In [42]:
#Agency related Information


import pandas as pd

In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import difflib
from difflib import SequenceMatcher
from scipy import stats
import string


In [3]:
df = pd.read_csv('Citywide_Payroll_Data__Fiscal_Year_.csv')

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df['Fiscal Year'] = pd.to_datetime(df['Fiscal Year'], format='%Y')

In [5]:
df.head()

Fiscal Year  Payroll Number                  Agency Name  Last Name  \
0  2016-01-01             NaN  DISTRICT ATTORNEY-MANHATTAN  ABA'AHMID   
1  2016-01-01             NaN  DISTRICT ATTORNEY-MANHATTAN    ABENSUR   
2  2016-01-01             NaN  DISTRICT ATTORNEY-MANHATTAN  ABOUNAOUM   
3  2016-01-01             NaN  DISTRICT ATTORNEY-MANHATTAN    ABRAHAM   
4  2016-01-01             NaN  DISTRICT ATTORNEY-MANHATTAN     ABRAMS   

  First Name Mid Init Agency Start Date Work Location Borough  \
0  RAHASHEEM        E        07/14/2003             MANHATTAN   
1   MARGARET      NaN        06/12/1995             MANHATTAN   
2     ANDREA        L        10/11/2011             MANHATTAN   
3   JONATHAN        J        12/01/2014             MANHATTAN   
4     JOSEPH      NaN        05/21/2015             MANHATTAN   

           Title Description Leave Status as of June 30  Base Salary  \
0        COMMUNITY ASSOCIATE                     ACTIVE      47678.0   
1  ADMINISTRATIVE ACCOUNTANT                     ACTIVE     119959.0   
2        COMMUNITY ASSOCIATE                   ON LEAVE      39966.0   
3   COMPUTER SYSTEMS MANAGER                     ACTIVE     116000.0   
4               COLLEGE AIDE                     CEASED          1.0   

   Pay Basis  Regular Hours  Regular Gross Paid  OT Hours  Total OT Paid  \
0  per Annum        1830.00            47166.03      2.25          57.28   
1  per Annum        1831.00           119042.50      0.00           0.00   
2  per Annum        1181.68            27452.62      1.00          21.34   
3  per Annum        1831.00           115104.57      0.00           0.00   
4   per Hour           0.00             3500.00      0.00           0.00   

   Total Other Pay  
0              0.0  
1              0.0  
2             33.0  
3              0.0  
4              0.0

In [6]:
df = df[df['Pay Basis']=='per Annum']
df.drop('Pay Basis', axis=1, inplace=True)

In [7]:
len(df) 

1988243

In [8]:
df['Agency Name'].value_counts()

DEPT OF ED PEDAGOGICAL           643361
POLICE DEPARTMENT                287554
DEPT OF ED PARA PROFESSIONALS    160818
FIRE DEPARTMENT                  106818
HRA/DEPT OF SOCIAL SERVICES       88258
                                  ...  
BROOKLYN COMMUNITY BOARD #10         12
BRONX COMMUNITY BOARD #1             12
BRONX COMMUNITY BOARD #11            11
STATEN ISLAND COMMUNITY BD #2         7
DISTRICTING COMMISSION                3
Name: Agency Name, Length: 158, dtype: int64

In [9]:
df.drop(['Agency Start Date', 'Leave Status as of June 30', 
         'Total Other Pay','Total OT Paid', 'Base Salary', 
         'OT Hours', 'Payroll Number', 'Mid Init', 'First Name', 'Last Name',
         'Regular Hours'], axis=1, inplace=True)

df['Title Description'].dropna(inplace=True)


In [10]:
df['Work Location Borough'] = df['Work Location Borough'].fillna('Unknown')

In [11]:
df = df[df['Work Location Borough'] != 'Unknown']

df = df[df['Regular Gross Paid']>20000]

In [12]:
df.head()

Fiscal Year                  Agency Name Work Location Borough  \
0  2016-01-01  DISTRICT ATTORNEY-MANHATTAN             MANHATTAN   
1  2016-01-01  DISTRICT ATTORNEY-MANHATTAN             MANHATTAN   
2  2016-01-01  DISTRICT ATTORNEY-MANHATTAN             MANHATTAN   
3  2016-01-01  DISTRICT ATTORNEY-MANHATTAN             MANHATTAN   
5  2016-01-01  DISTRICT ATTORNEY-MANHATTAN             MANHATTAN   

             Title Description  Regular Gross Paid  
0          COMMUNITY ASSOCIATE            47166.03  
1    ADMINISTRATIVE ACCOUNTANT           119042.50  
2          COMMUNITY ASSOCIATE            27452.62  
3     COMPUTER SYSTEMS MANAGER           115104.57  
5  ASSISTANT DISTRICT ATTORNEY            71967.34

In [13]:
#df = df[df['Pay Basis'] != 'Prorated Annual']

In [14]:
df.isna().sum()

Fiscal Year               0
Agency Name               0
Work Location Borough     0
Title Description        46
Regular Gross Paid        0
dtype: int64

In [15]:
df.columns

Index(['Fiscal Year', 'Agency Name', 'Work Location Borough',
       'Title Description', 'Regular Gross Paid'],
      dtype='object')

In [16]:
def remove_punctuations(words):
    buffer = []
    punc = string.punctuation
    for w in str(words):
        if w not in punc:
            buffer.append(w)
    return ''.join(buffer).lstrip()

In [17]:
df['Title Description'] = df['Title Description'].apply(remove_punctuations)

In [18]:
df['Work Location Borough'].value_counts()

MANHATTAN        910244
QUEENS           208671
BROOKLYN         191720
BRONX            101273
OTHER             55006
RICHMOND          27552
WESTCHESTER        2348
ULSTER             1365
Manhattan          1301
Bronx               856
Queens              600
SULLIVAN            568
DELAWARE            382
PUTNAM              190
SCHOHARIE           138
Richmond            101
NASSAU               85
DUTCHESS             65
ALBANY               54
GREENE               40
WASHINGTON DC        31
ORANGE                8
Name: Work Location Borough, dtype: int64

In [19]:
#df.nunique()

In [20]:
df.groupby(df['Title Description'])['Regular Gross Paid'].mean().sort_values(ascending=False)

Title Description
PENSION INVESTMENT ADVISOR                275666.675000
DIRECTOR OF  INVESTMENTS                  252521.213571
FIRST DEPUTY MAYOR                        239581.494000
CHIEF OF STAFF  MAYORS OFFICE             238149.780000
DIRECTOR OF CRIMINAL JUSTICE              237517.730000
                                              ...      
EXECUTIVE ASST TO THE COMMISSIONER         21970.930000
CONFIDENTIAL ASST TO THE EXECUTIVE DIR     20942.480000
ASSISTANT PRINCIPAL ADMIN                  20819.600000
SIGN LANGUAGE INTERPRETER                  20809.200000
LEGISLATIVE FISCAL OFFICER                 20514.050000
Name: Regular Gross Paid, Length: 1476, dtype: float64

In [22]:
df.pivot_table(values = ['Regular Gross Paid'], 
               index=['Work Location Borough', 'Agency Name'])

Regular Gross Paid
Work Location Borough Agency Name                                       
ALBANY                CITY COUNCIL                         110315.330000
                      DEPT OF ENVIRONMENT PROTECTION        61589.268261
                      OFFICE OF THE MAYOR                   72679.280000
BRONX                 ADMIN FOR CHILDREN'S SVCS             57399.759775
                      ADMIN TRIALS AND HEARINGS             57469.962692
...                                                                  ...
WASHINGTON DC         OFFICE OF THE MAYOR                   90322.894783
WESTCHESTER           DEPARTMENT OF EDUCATION ADMIN         54999.586667
                      DEPARTMENT OF INVESTIGATION           63840.315000
                      DEPT OF ENVIRONMENT PROTECTION        70405.787768
                      HOUSING PRESERVATION & DVLPMNT        41152.200000

[307 rows x 1 columns]

In [39]:
title_loc_salary = df.pivot_table(values = ['Regular Gross Paid'], 
               index=['Title Description', 'Work Location Borough'])
title_loc_salary

Regular Gross Paid
Title Description                         Work Location Borough                    
12 MONTH SPECIAL EDUCATION ASST PRINCIPAL MANHATTAN                   126253.908194
                                          OTHER                       128237.335714
12 MONTH SPECIAL EDUCATION SUPERVISOR     MANHATTAN                   127228.849213
                                          OTHER                       132011.055000
ACCOUNTANT                                BRONX                        68681.085556
...                                                                             ...
YOUTH DEVELOPMENT SPECIALIST              MANHATTAN                    47267.565676
                                          QUEENS                       44899.516667
nan                                       MANHATTAN                    33408.534000
                                          QUEENS                       56078.945000
                                          RICHMOND                     70729.858387

[3804 rows x 1 columns]

In [40]:
title_loc_salary.reindex(title_loc_salary['Regular Gross Paid'].sort_values(ascending=False).index)

Regular Gross Paid
Title Description                      Work Location Borough                    
PENSION INVESTMENT ADVISOR             MANHATTAN                   275666.675000
DIRECTOR OF  INVESTMENTS               MANHATTAN                   252521.213571
PRESIDENT                              QUEENS                      244962.353636
FIRST DEPUTY MAYOR                     MANHATTAN                   239581.494000
CHIEF OF STAFF  MAYORS OFFICE          MANHATTAN                   238149.780000
...                                                                          ...
ASSISTANT TEACHER                      QUEENS                       21540.540000
CONFIDENTIAL ASST TO THE EXECUTIVE DIR MANHATTAN                    20942.480000
ASSISTANT PRINCIPAL ADMIN              MANHATTAN                    20819.600000
SIGN LANGUAGE INTERPRETER              MANHATTAN                    20809.200000
LEGISLATIVE FISCAL OFFICER             MANHATTAN                    20514.050000

[3804 rows x 1 columns]

In [25]:
df.groupby(['Title Description', 'Work Location Borough'])['Regular Gross Paid'].mean()

Title Description                          Work Location Borough
12 MONTH SPECIAL EDUCATION ASST PRINCIPAL  MANHATTAN                126253.908194
                                           OTHER                    128237.335714
12 MONTH SPECIAL EDUCATION SUPERVISOR      MANHATTAN                127228.849213
                                           OTHER                    132011.055000
ACCOUNTANT                                 BRONX                     68681.085556
                                                                        ...      
YOUTH DEVELOPMENT SPECIALIST               MANHATTAN                 47267.565676
                                           QUEENS                    44899.516667
nan                                        MANHATTAN                 33408.534000
                                           QUEENS                    56078.945000
                                           RICHMOND                  70729.858387
Name: Regular Gross Paid, Length:

In [26]:
df.groupby('Title Description').mean().sort_values('Regular Gross Paid', ascending=False)

Regular Gross Paid
Title Description                                         
PENSION INVESTMENT ADVISOR                   275666.675000
DIRECTOR OF  INVESTMENTS                     252521.213571
FIRST DEPUTY MAYOR                           239581.494000
CHIEF OF STAFF  MAYORS OFFICE                238149.780000
DIRECTOR OF CRIMINAL JUSTICE                 237517.730000
...                                                    ...
EXECUTIVE ASST TO THE COMMISSIONER            21970.930000
CONFIDENTIAL ASST TO THE EXECUTIVE DIR        20942.480000
ASSISTANT PRINCIPAL ADMIN                     20819.600000
SIGN LANGUAGE INTERPRETER                     20809.200000
LEGISLATIVE FISCAL OFFICER                    20514.050000

[1476 rows x 1 columns]

In [27]:
df.groupby('Fiscal Year')['Regular Gross Paid'].describe().pct_change()[['mean','std', 'min', '25%','50%','75%','max']]

 #Total salary costs(fulltime workers) and change from (total and boroughs)

mean       std       min       25%       50%       75%  \
Fiscal Year                                                               
2014-01-01        NaN       NaN       NaN       NaN       NaN       NaN   
2015-01-01  -0.006766 -0.242971 -0.000289  0.116648  0.095050  0.034259   
2016-01-01   0.026180  0.022648  0.000034  0.022417  0.008166  0.048765   
2017-01-01   0.039816  0.057118 -0.000048  0.036127  0.036258  0.055344   
2018-01-01   0.002758 -0.005624  0.000063 -0.006323  0.007892 -0.009526   
2019-01-01   0.073473  0.126235 -0.000056  0.052806  0.060336  0.080248   

                  max  
Fiscal Year            
2014-01-01        NaN  
2015-01-01   1.478669  
2016-01-01   0.377682  
2017-01-01  -0.460938  
2018-01-01   0.017056  
2019-01-01   0.356493

In [28]:
df.groupby(['Work Location Borough'])['Regular Gross Paid'].describe().pct_change()[['mean','std', 'min', '25%','50%','75%','max']]
#+'Fiscal Year', 

mean       std       min       25%       50%  \
Work Location Borough                                                     
ALBANY                      NaN       NaN       NaN       NaN       NaN   
BRONX                 -0.114329 -0.143017 -0.213731 -0.181145 -0.214714   
BROOKLYN               0.011666  0.042052  0.000031  0.003465  0.001676   
Bronx                 -0.105076  0.199548  0.000255 -0.196368 -0.130072   
DELAWARE               0.102858 -0.382602  0.005814  0.549047  0.177452   
DUTCHESS               0.166897  0.226929  0.387663  0.037558  0.299332   
GREENE                -0.026904 -0.205653  0.651660  0.009157 -0.077905   
MANHATTAN             -0.020567  0.512957 -0.566343 -0.175667 -0.011259   
Manhattan             -0.020948  0.219838  0.001181 -0.118709 -0.104297   
NASSAU                -0.159902 -0.633075  0.183706  0.163262 -0.026421   
ORANGE                -0.065220 -0.675688  1.061063  0.082216 -0.108922   
OTHER                  0.831643  6.869167 -0.590590  0.706542  0.952003   
PUTNAM                -0.373387 -0.582532  0.059756 -0.369107 -0.445801   
QUEENS                 0.075640  0.932234 -0.056349 -0.185834  0.106015   
Queens                 0.052058  0.384160  0.020703 -0.073326 -0.054272   
RICHMOND              -0.034383 -0.332387 -0.020122  0.151184  0.103363   
Richmond               0.077944  0.346368  0.090372  0.070190 -0.054465   
SCHOHARIE             -0.097160 -0.466470  0.631383  0.084473 -0.019364   
SULLIVAN               0.070011  0.543565 -0.416883 -0.010203  0.021531   
ULSTER                 0.051063 -0.138386 -0.035100  0.041403  0.130464   
WASHINGTON DC          0.252459  1.038858  0.055970 -0.059894  0.221843   
WESTCHESTER           -0.250641 -0.496243 -0.043500 -0.007010 -0.249120   

                            75%       max  
Work Location Borough                      
ALBANY                      NaN       NaN  
BRONX                  0.009675  0.994803  
BROOKLYN               0.000000  0.711145  
Bronx                 -0.227923 -0.608241  
DELAWARE               0.095682 -0.180642  
DUTCHESS               0.149663 -0.008035  
GREENE                -0.001700 -0.156905  
MANHATTAN              0.094310  2.630292  
Manhattan             -0.026029 -0.611147  
NASSAU                -0.210298 -0.573148  
ORANGE                -0.165345 -0.243749  
OTHER                  0.997917  3.523693  
PUTNAM                -0.388608 -0.553106  
QUEENS                 0.210690  4.437080  
Queens                -0.005811 -0.707159  
RICHMOND               0.005845  0.547170  
Richmond              -0.024424 -0.377954  
SCHOHARIE             -0.172131 -0.225405  
SULLIVAN               0.164342  0.498341  
ULSTER                 0.081540 -0.164764  
WASHINGTON DC          0.518948 -0.048260  
WESTCHESTER           -0.362580  0.091196

In [29]:
df.groupby(['Title Description','Work Location Borough'])['Regular Gross Paid'].mean()

Title Description                          Work Location Borough
12 MONTH SPECIAL EDUCATION ASST PRINCIPAL  MANHATTAN                126253.908194
                                           OTHER                    128237.335714
12 MONTH SPECIAL EDUCATION SUPERVISOR      MANHATTAN                127228.849213
                                           OTHER                    132011.055000
ACCOUNTANT                                 BRONX                     68681.085556
                                                                        ...      
YOUTH DEVELOPMENT SPECIALIST               MANHATTAN                 47267.565676
                                           QUEENS                    44899.516667
nan                                        MANHATTAN                 33408.534000
                                           QUEENS                    56078.945000
                                           RICHMOND                  70729.858387
Name: Regular Gross Paid, Length:

In [30]:
df.groupby('Title Description').mean().sort_values('Regular Gross Paid', ascending=False)

Regular Gross Paid
Title Description                                         
PENSION INVESTMENT ADVISOR                   275666.675000
DIRECTOR OF  INVESTMENTS                     252521.213571
FIRST DEPUTY MAYOR                           239581.494000
CHIEF OF STAFF  MAYORS OFFICE                238149.780000
DIRECTOR OF CRIMINAL JUSTICE                 237517.730000
...                                                    ...
EXECUTIVE ASST TO THE COMMISSIONER            21970.930000
CONFIDENTIAL ASST TO THE EXECUTIVE DIR        20942.480000
ASSISTANT PRINCIPAL ADMIN                     20819.600000
SIGN LANGUAGE INTERPRETER                     20809.200000
LEGISLATIVE FISCAL OFFICER                    20514.050000

[1476 rows x 1 columns]

In [31]:
# cat_paybasis = pd.get_dummies(df['Pay Basis'])
# df2 = pd.concat([df, cat_paybasis], axis=1)
# df2.drop('Pay Basis', axis=1, inplace=True)

# def similar(a, b):
#     return SequenceMatcher(None, a, b).ratio()

# q_low = df2["Regular Gross Paid"].quantile(0.05)
# q_hi  = df2["Regular Gross Paid"].quantile(0.99)
# df3 = df2[(df2["Regular Gross Paid"] < q_hi) & (df2["Regular Gross Paid"] > q_low)]

In [32]:
#Cost Structure boroughs
df.groupby('Work Location Borough').describe()

Regular Gross Paid                              \
                                   count          mean           std   
Work Location Borough                                                  
ALBANY                              54.0  72137.539815  30859.562978   
BRONX                           101273.0  63890.131128  26446.116325   
BROOKLYN                        191720.0  64635.470369  27558.227939   
Bronx                              856.0  57843.814930  33057.409217   
DELAWARE                           382.0  63793.506021  20409.580110   
DUTCHESS                            65.0  74440.449385  25041.109156   
GREENE                              40.0  72437.710250  19891.317946   
MANHATTAN                       910244.0  70947.910235  30094.702593   
Manhattan                         1301.0  69461.701991  36710.672769   
NASSAU                              85.0  58354.618471  13470.053137   
ORANGE                               8.0  54548.705000   4368.498368   
OTHER                            55006.0  99913.744283  34376.444991   
PUTNAM                             190.0  62607.228000  14351.072343   
QUEENS                          208671.0  67342.835224  27729.632556   
Queens                             600.0  70848.553483  38382.245593   
RICHMOND                         27552.0  68412.544122  25624.497577   
Richmond                           101.0  73744.899901  34500.007523   
SCHOHARIE                          138.0  66579.827029  18406.782177   
SULLIVAN                           568.0  71241.172553  28412.062443   
ULSTER                            1365.0  74878.977883  24480.233427   
WASHINGTON DC                       31.0  93782.871935  49911.708665   
WESTCHESTER                       2348.0  70277.077994  25143.383478   

                                                                      \
                            min         25%         50%          75%   
Work Location Borough                                                  
ALBANY                 25437.20  52458.5050   73047.225   83516.2625   
BRONX                  20000.48  42955.9000   57362.960   84324.2500   
BROOKLYN               20001.11  43104.7300   57459.090   84324.2500   
Bronx                  20006.21  34640.3325   49985.265   65104.8500   
DELAWARE               20122.52  53659.5100   58855.245   71334.2300   
DUTCHESS               27923.28  55674.8300   76472.520   82010.3500   
GREENE                 46119.76  56184.6400   70514.950   81870.9200   
MANHATTAN              20000.14  46314.8400   69720.995   89592.1400   
Manhattan              20023.76  40816.8700   62449.310   87260.1700   
NASSAU                 23702.24  47480.7000   60799.350   68909.5000   
ORANGE                 48851.82  51384.3775   54176.940   57515.6250   
OTHER                  20000.43  87689.5800  105753.560  114911.4400   
PUTNAM                 21195.58  55322.7500   58608.530   70255.9650   
QUEENS                 20001.23  45041.9200   64821.900   85058.2200   
Queens                 20415.32  41739.1650   61303.880   84563.9625   
RICHMOND               20004.53  48049.4400   67640.410   85058.2200   
Richmond               21812.37  51422.0400   63956.360   82980.7500   
SCHOHARIE              35584.32  55765.8325   62717.895   68697.2000   
SULLIVAN               20749.82  55196.8750   64068.270   79987.0500   
ULSTER                 20021.51  57482.2100   72426.900   86509.2100   
WASHINGTON DC          21142.12  54039.3900   88494.280  131402.9600   
WESTCHESTER            20222.44  53660.5500   66448.550   83758.8550   

                                  
                             max  
Work Location Borough             
ALBANY                 146481.64  
BRONX                  292201.96  
BROOKLYN               500000.00  
Bronx                  195879.42  
DELAWARE               160495.35  
DUTCHESS               159205.70  
GREENE                 134225.46  
MANHATTAN              487277.63  
Manhattan              189479.42  


In [33]:
df.groupby('Title Description').mean().sort_values('Regular Gross Paid', ascending=False)

Regular Gross Paid
Title Description                                         
PENSION INVESTMENT ADVISOR                   275666.675000
DIRECTOR OF  INVESTMENTS                     252521.213571
FIRST DEPUTY MAYOR                           239581.494000
CHIEF OF STAFF  MAYORS OFFICE                238149.780000
DIRECTOR OF CRIMINAL JUSTICE                 237517.730000
...                                                    ...
EXECUTIVE ASST TO THE COMMISSIONER            21970.930000
CONFIDENTIAL ASST TO THE EXECUTIVE DIR        20942.480000
ASSISTANT PRINCIPAL ADMIN                     20819.600000
SIGN LANGUAGE INTERPRETER                     20809.200000
LEGISLATIVE FISCAL OFFICER                    20514.050000

[1476 rows x 1 columns]

In [41]:
#Total Cost Structure Agencies
df.groupby('Agency Name').sum().sort_values('Regular Gross Paid', ascending=False)/1000000

Regular Gross Paid
Agency Name                                      
DEPT OF ED PEDAGOGICAL               40909.485991
POLICE DEPARTMENT                    18097.325968
FIRE DEPARTMENT                       6231.503590
DEPT OF ED PARA PROFESSIONALS         4323.200210
DEPARTMENT OF CORRECTION              3835.876042
...                                           ...
MANHATTAN COMMUNITY BOARD #9             0.650765
BRONX COMMUNITY BOARD #11                0.641650
QUEENS COMMUNITY BOARD #1                0.641292
STATEN ISLAND COMMUNITY BD #2            0.468159
BROOKLYN COMMUNITY BOARD #9              0.421069

[152 rows x 1 columns]

In [35]:
len(df)

1502598

In [36]:
df.head()

Fiscal Year                  Agency Name Work Location Borough  \
0  2016-01-01  DISTRICT ATTORNEY-MANHATTAN             MANHATTAN   
1  2016-01-01  DISTRICT ATTORNEY-MANHATTAN             MANHATTAN   
2  2016-01-01  DISTRICT ATTORNEY-MANHATTAN             MANHATTAN   
3  2016-01-01  DISTRICT ATTORNEY-MANHATTAN             MANHATTAN   
5  2016-01-01  DISTRICT ATTORNEY-MANHATTAN             MANHATTAN   

             Title Description  Regular Gross Paid  
0          COMMUNITY ASSOCIATE            47166.03  
1    ADMINISTRATIVE ACCOUNTANT           119042.50  
2          COMMUNITY ASSOCIATE            27452.62  
3     COMPUTER SYSTEMS MANAGER           115104.57  
5  ASSISTANT DISTRICT ATTORNEY            71967.34